<a href="https://colab.research.google.com/github/BondaiKa/lane_line_recognition/blob/main/vl_100_lane_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print(sys.version)

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


In [2]:
import os
from google.colab import drive
import numpy as np
import glob
import json

import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf

from typing import NamedTuple, Tuple, List

%matplotlib inline

!pip install requests
!pip install unrar
!pip install -Uqq ipdb
import ipdb

     |████████████████████████████████| 792 kB 13.1 MB/s 
     |████████████████████████████████| 374 kB 59.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.24 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.0 which is incompatible.


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unrar x -Y "/content/drive/My Drive/Ilmenau/dataset.rar" "/tmp/"

Streaming output truncated to the last 5000 lines.
Extracting  /tmp/dataset/VIL100/Json/25_Road015_Trim006_frames/00150.jpg.json      99%  OK 
Extracting  /tmp/dataset/VIL100/Json/25_Road015_Trim006_frames/00153.jpg.json      99%  OK 
Extracting  /tmp/dataset/VIL100/Json/25_Road015_Trim006_frames/00156.jpg.json      99%  OK 
Extracting  /tmp/dataset/VIL100/Json/25_Road015_Trim006_frames/00159.jpg.json      99%  OK 
Extracting  /tmp/dataset/VIL100/Json/25_Road015_Trim006_frames/00162.jpg.json      99%  OK 
Extracting  /tmp/dataset/VIL100/Json/25_Road015_Trim006_frames/00165.jpg.json      99%  OK 
Extracting  /tmp/dataset/VIL100/Json/25_Road015_Trim006_frames/00168.jpg.json      99%  OK 
Extracting  /tmp/dataset/VIL100/Json/25_Road015_Trim006_frames/00171.jpg.json      99%  OK 
Extracting  /tmp/dataset/VIL100/Json/25_Road015_Trim006_frames/00174.jpg.json      99%  OK 
Extracting  /tmp/dataset/VIL100/Json/25_

In [5]:
BASE_DIR = "/tmp/dataset/VIL100/"
IMAGE_PATH = BASE_DIR + "JPEGImages/"
JSON_PATH = BASE_DIR + "Json/"
INPUT_SHAPE = (1280,960)

print(IMAGE_PATH)
print(JSON_PATH)

/tmp/dataset/VIL100/JPEGImages/
/tmp/dataset/VIL100/Json/


In [6]:
# from google.colab import files
# !zip -r "/tmp/dataset/VIL100/JPEGImages/0_Road029_Trim004_frames.zip" "/tmp/dataset/VIL100/JPEGImages/0_Road029_Trim004_frames/"
# files.download("/tmp/dataset/VIL100/JPEGImages/0_Road029_Trim004_frames.zip")

In [7]:
# from google.colab import files
# !zip -r "/tmp/dataset/VIL100/Json/0_Road029_Trim004_frames.zip" "/tmp/dataset/VIL100/Json/0_Road029_Trim004_frames/"
# files.download("/tmp/dataset/VIL100/Json/0_Road029_Trim004_frames.zip")

In [8]:
images = glob.glob(IMAGE_PATH+'/*/*.jpg')
json_files = glob.glob(JSON_PATH+'/*/*.json')
print(images)
print(json_files)

['/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00084.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00042.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00087.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00111.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00117.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00276.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00162.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00270.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00075.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00060.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00234.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00243.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00255.jpg', '/tmp/dataset/VIL100/JPEGImages/7_Road003_Trim001_frames/00108.jpg', '/tmp/dataset/VIL100/JPEGImages/7

In [9]:
def calculate_perspective_transform_matrix(width: int, height: int, reverse_flag=False) -> Tuple[
    np.ndarray]:
    """
    Calculate transformation matrix for perspective transformation
    :param width: frame width
    :param height: frame height
    :param reverse_flag: create reverse matrix for reverting to initial frame
    :return: matrix for transformation the frame
    """
    # TODO @Karim: check on real Audi Q2 input frame
    high_left_crd, high_right_crd = (550, 530), (700, 530)
    down_left_crd, down_right_crd, = (0, height - 150), (width, height - 150)

    initial_matrix = np.float32([[high_left_crd, high_right_crd,
                                  down_left_crd, down_right_crd]])
    final_matrix = np.float32([[(0, 0), (width, 0), (0, height), (width, height)]])

    return cv2.getPerspectiveTransform(initial_matrix, final_matrix) \
        if not reverse_flag else cv2.getPerspectiveTransform(final_matrix, initial_matrix)


def transform_frame(frame: np.ndarray, width: int, height: int, reverse_flag=False) -> np.ndarray:
    """
    Perform perspective transformation
    :param frame: frame
    :param width: frame width
    :param height: frame height
    :param reverse_flag: cancel perspective transformation
    :return: changed (un)transformed frame
    """
    if not reverse_flag:
        initial_matrix = calculate_perspective_transform_matrix(width, height)
        frame = cv2.warpPerspective(frame, initial_matrix, dsize=(width, height))
    else:
        final_matrix = calculate_perspective_transform_matrix(width, height, reverse_flag=True)
        frame = cv2.warpPerspective(frame, final_matrix, dsize=(width, height))
    return frame

In [10]:

# FRAME_WIDTH, FRAME_HEIGHT = 1280, 960
# BATCH_SIZE = 64
# TRAIN_DATA_DIR = "/tmp/dataset/VIL100/JPEGImages/"

# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     TRAIN_DATA_DIR,
#     rescale = 1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2) # set validation split

# train_generator = train_datagen.flow_from_directory(
#     TRAIN_DATA_DIR,
#     rescale = 1./255,
#     target_size=(FRAME_HEIGHT, FRAME_WIDTH),
#     batch_size=batch_size,
#     class_mode='binary',
#     subset='training') # set as training data

# validation_generator = train_datagen.flow_from_directory(
#     TRAIN_DATA_DIR, # same directory as training data
#     rescale = 1./255,
#     target_size=(FRAME_HEIGHT, FRAME_WIDTH),
#     batch_size=BATCH_SIZE,
#     class_mode='binary',
#     subset='validation') # set as validation data

# model.fit_generator(
#     TRAIN_DATA_DIR,
#     rescale = 1./255,
#     steps_per_epoch = train_generator.samples // batch_size,
#     validation_data = validation_generator, 
#     validation_steps = validation_generator.samples // batch_size,
#     epochs = nb_epochs)

In [182]:
from tensorflow.keras.utils import Sequence
from typing import Tuple, List, Dict, Iterable
from skimage.io import imread
from skimage.transform import resize
import logging
import math
from typing import Optional
import random

log = logging.getLogger(__name__)


def one_hot_list_encoder(target_class_idx, num_classes) -> List[int]:
    """one-hot list encoder"""
    target_vector = [0] * num_classes
    target_vector[target_class_idx] = 1
    return target_vector


class SimpleFrameGenerator(Sequence):
    """Sequence of frames generator

    Usage for training NN that could process independent
    frames without context window etc
    """

    def __init__(self,
                 num_type_of_lines=2,
                 max_num_points=91,
                 max_lines_per_frame=6,
                 rescale=1 / 255.,  # TODO @Karim: include and use later
                 batch_size: int = 8,
                 target_shape: Tuple[int, int] = (1280, 960),
                 shuffle: bool = False,
                 nb_channel: int = 3,  # TODO: Use rgb later
                 files: Optional[List[str]] = None,
                 json_files: Optional[List[str]] = None):
        """
        :param subset: training or validation data
        :param max_lines_per_frame: maxinum number of lines per frame
        :param max_num_points: maximum number of points un one polyline
        :param num_type_of_lines: number of possible lines on road
        :param rescale:
        :param batch_size: batch size of the dataset
        :param target_shape: final size for NN input
        :param shuffle: shuffle flag of frames sequences
        :param split: split dataset to train/test
        :param nb_channel: grayscaled or RGB frames
        :param frame_glob_path: glob pattern of frames
        :param json_glob_path: glob pattern path of jsons
        """
        self.max_lines_per_frame = max_lines_per_frame
        self.max_num_points = max_num_points
        self.num_type_of_lines = num_type_of_lines
        self.rescale = rescale
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.target_shape = target_shape
        self.nb_channel = nb_channel
        self.files = files
        self.json_files = json_files
        self.files_count = len(self.files)

        if shuffle:
            temp = list(zip(self.files, self.json_files))
            random.shuffle(temp)
            self.files, self.json_files = zip(*temp)

    def __len__(self):
        return math.ceil(self.files_count / self.batch_size)

    def __get_polyline_with_label(self, lane: dict) -> np.ndarray:
        """Get array from points list"""
        res = np.array(
            lane["points"]).flatten()
        res = np.pad(res, pad_width=(0, self.max_num_points * 2 - res.shape[0]))
        res = np.hstack((res, one_hot_list_encoder(lane.get('label', 0), self.num_type_of_lines)))
        return res

    def __get_polyline_from_file(self, json_path: str) -> np.ndarray:
        """
        Get all Polygonal chains from json file
        :param json_path: path of json file
        :return: right points for frame
        """
        with open(json_path) as f:
            polylines: List[Dict[str, int]] = json.load(f)["annotations"]["lane"]
            if polylines:
            # TODO @Karim: check another params in json files like "occlusion"
              res = np.vstack(list(map(lambda lane: self.__get_polyline_with_label(lane=lane), polylines)))
              empty_lines = np.zeros(
                  shape=(self.max_lines_per_frame - res.shape[0], self.max_num_points * 2 + self.num_type_of_lines),dtype=np.float32)
              return np.vstack((res, empty_lines)).flatten() #todo: rewrite to return flatten 1, 284 * 6 without flatten
            else: 
              return np.zeros(shape=(self.max_lines_per_frame * (self.max_num_points * 2 + self.num_type_of_lines)))

    def __getitem__(self, idx) -> Tuple[np.ndarray, np.ndarray]:
        batch_frames_path = self.files[idx * self.batch_size:
                                       (idx + 1) * self.batch_size]
        batch_json_path = self.json_files[idx * self.batch_size:
                                          (idx + 1) * self.batch_size]
        polylines = np.array(list(map(lambda x: self.__get_polyline_from_file(x),
                                      batch_json_path))) #,dtype=np.float32
        return np.array([
            resize(imread(file_name) * self.rescale, self.target_shape) for file_name in batch_frames_path]), polylines


class SimpleFrameDataGen:
    TRAINING = 'training'
    VALIDATION = 'validation'

    __reverse_dataset_type = {
        TRAINING: VALIDATION,
        VALIDATION: TRAINING
    }
    __dataset = {}

    def __init__(self,
                 rescale=1 / 255.,
                 validation_split: Optional[float] = None,
                 frame_glob_path: str = "",
                 json_glob_path: str = ""):
        """
        :param validation_split: split for train/validation sets
        :param rescale:
        :param frame_glob_path: glob pattern of frames
        :param json_glob_path: glob pattern path of jsons
        """
        self.rescale = rescale
        self.validation_split = validation_split

        self.__frame_glob_path = frame_glob_path
        self.__json_glob_path = json_glob_path

    def flow_from_directory(self, subset: str = TRAINING,
                            shuffle: bool = True, *args, **kwargs) -> SimpleFrameGenerator:
        """
        Get generator for subset
        :param subset: 'training' or 'validation'
        :param shuffle: flag for shuffling
        :param args: args for specific dataset
        :param kwargs: kwargs for specific dataset
        :return: Specific generator for specific subset
        """

        files = sorted(glob.glob(self.__frame_glob_path))
        json_files = sorted(glob.glob(self.__json_glob_path))
        files_count = len(files)
        json_files_count = len(json_files)

        if files_count != json_files_count:
            log.error(f"Dataset files error"
                      f"Number of frames: ({files_count}). "
                      f"Number of jsons({json_files_count}")
            raise FileNotFoundError(
                f"Numbers of frames and jsons are not equal!")

        if not self.__reverse_dataset_type.get(subset):
            log.error(f'Wrong subset value: "{subset}"')
            raise ValueError(f'Wrong type of subset - {subset}. '
                             f'Available types: {self.__reverse_dataset_type.keys()}')

        if self.validation_split and 0.0 < self.validation_split < 1.0:
            split = int(files_count * (1 - self.validation_split))
            if subset == self.TRAINING:
                files = files[:split]
                json_files = json_files[:split]
            else:
                files = files[split:]
                json_files = json_files[split:]

        return SimpleFrameGenerator(rescale=self.rescale,
                                    files=files,
                                    shuffle=shuffle,
                                    json_files=json_files,
                                    *args, **kwargs)

In [183]:
data_gen = SimpleFrameDataGen(validation_split=0.2, frame_glob_path=IMAGE_PATH+'/*/*.jpg', json_glob_path=JSON_PATH+'/*/*.json')
train_generator = data_gen.flow_from_directory(subset='training', shuffle=True)
validation_generator = data_gen.flow_from_directory(subset='validation', shuffle=True)


In [184]:
# for image_polylines in validation_generator:
#   print((image_polylines[1]))

In [185]:
from tensorflow.keras import layers, Model


def build_model(output_shape, input_shape=(1280, 960, 3)):

    # Input layer returning the `input_img` tensor
    input_img = layers.Input(shape=input_shape)

    # START YOUR CODE HERE ### (~7 LOC)

    # First block: 16 convolution filters (3x3), relu, maxpool
    conv_1 = layers.Conv2D(16, (3, 3), activation='relu')(input_img)
    max_pool_1 = layers.MaxPooling2D()(conv_1)
    # Second block: 32 convolution filters (3x3), relu, maxpool
    conv_2 = layers.Conv2D(32, (3, 3), activation='relu')(max_pool_1)
    max_pool_2 = layers.MaxPooling2D()(conv_2)
    # Third block: 64 filters (3x3), relu
    conv_3 = layers.Conv2D(64, (3, 3), activation='relu')(max_pool_2)
    # GlobalMaxPool
    global_max_pool = layers.GlobalMaxPool2D()(conv_3)
    # Create the output layer with `num_classes` nodes and softmax activation
    output = layers.Dense(output_shape)(global_max_pool)

    ### END YOUR CODE HERE ###

    # Build the model by defining the input and output tensors
    model = Model(input_img, output)

    return model

In [186]:
model = build_model(output_shape=184*6)
print(model.summary())

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1280, 960, 3)]    0         
                                                                 
 conv2d_18 (Conv2D)          (None, 1278, 958, 16)     448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 639, 479, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 637, 477, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 318, 238, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 316, 236, 64)      1849

In [187]:
# tf.experimental.numpy.experimental_enable_numpy_behavior()


In [188]:
from tensorflow.keras.optimizers import Adam

learning_rate = 3e-4

model.compile(loss='mse',
              optimizer=Adam(learning_rate=learning_rate),
              metrics=['accuracy'])
# tf.keras.metrics.MeanSquaredLogarithmicError()

In [ ]:
history = model.fit(train_generator,
                    epochs=10,
                    verbose=2,
                    validation_data=validation_generator)

Epoch 1/10
